# 간단한 파이프라인 예제 실습


## 0) 최신 KFP v2 환경 설정

In [ ]:
import kfp
print("KFP 버전:", kfp.__version__)  # 2.x 이상이어야 함

from kfp import dsl
from kfp.dsl import Input, Output, Dataset, Model
from typing import get_type_hints

# 카카오클라우드 공식 헬퍼 (타입 힌트 자동 해석)
def _eval_annotations(fn):
    try:
        fn.__annotations__ = get_type_hints(fn, globalns=fn.__globals__)
    except:
        pass
    return fn

def component2(**kwargs):
    def _dec(fn):
        fn = _eval_annotations(fn)
        return dsl.component(**kwargs)(fn)
    return _dec

def pipeline2(**kwargs):
    def _dec(fn):
        fn = _eval_annotations(fn)
        return dsl.pipeline(**kwargs)(fn)
    return _dec

print("최신 KFP v2 환경 설정 완료!")

## 1) 컴포넌트 정의 (최신 데코레이터 방식)

In [ ]:
# 최신 베이스 이미지 (파이썬 3.11 추천)
BASE_IMAGE = "python:3.11-slim"

@component2(base_image=BASE_IMAGE)
def add_op(a: float, b: float) -> float:
    """두 수를 더하는 컴포넌트"""
    result = a + b
    print(f"{a} + {b} = {result}")
    return result

@component2(base_image=BASE_IMAGE)
def multiply_op(x: float, y: float) -> float:
    """두 수를 곱하는 컴포넌트"""
    result = x * y
    print(f"{x} × {y} = {result}")
    return result

print("컴포넌트 정의 완료!")

## 2) 파이프라인 정의 (최신 데코레이터 방식)

In [ ]:
@pipeline2(
    name="Addition and Multiplication Pipeline",
    description="두 수를 더하고, 그 결과를 세 번째 수와 곱하는 간단한 파이프라인"
)
def math_pipeline(
    a: float = 1.0,
    b: float = 2.0,
    c: float = 3.0
):
    """덧셈 → 곱셈 파이프라인"""
    add_task = add_op(a=a, b=b)
    multiply_task = multiply_op(x=add_task.output, y=c)

print("파이프라인 정의 완료!")

## 3) 파이프라인 컴파일 (YAML 저장)

In [ ]:
from kfp.compiler import Compiler

pipeline_file = "math_pipeline_latest.yaml"
Compiler().compile(math_pipeline, pipeline_file)

print(f"파이프라인 컴파일 완료! → {pipeline_file}")
print("이 파일을 Kubeflow Pipelines UI에 업로드하세요!")

## 4) 파이프라인 실행 (두 가지 방법)

In [ ]:
import os
from kakaocloud_kbm import KbmPipelineClient

# 카카오클라우드 접속 정보 (실제 값으로 바꾸세요)
os.environ["KUBEFLOW_HOST"] = "https://대시보드 접속한 로드밸런서의 Public IP/"
os.environ["KUBEFLOW_USERNAME"] = "Kubeflow 대시보드 접속 아이디(이메일)"
os.environ["KUBEFLOW_PASSWORD"] = "Kubeflow 대시보드 접속 비밀번호"

client = KbmPipelineClient(verify_ssl=False)

# 방법 1: 코드로 바로 실행 (테스트용)
client.create_run_from_pipeline_func(
    math_pipeline,
    experiment_name="Simple Math Test",
    run_name="run-a10-b20-c5",
    arguments={"a": 10, "b": 20, "c": 5}  # 결과: (10+20)×5 = 150
)

print("파이프라인 실행 시작! Kubeflow 대시보드에서 확인하세요")
print("또는 math_pipeline_latest.yaml 파일을 UI에 업로드해서 실행하세요!")